In [153]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
from tqdm import tqdm
from functools import reduce

In [154]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

In [155]:
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

In [156]:
from utils.geometrics import haversine_distance

In [157]:
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [158]:
df_weather_by_day = pd.read_parquet('../data/02_intermediate/weather_by_day.parquet')
df_weather_by_day.head()

,date,temperature_2m_min,temperature_2m_max,temperature_2m,apparent_temperature,precipitation,cloud_cover,dew_point_2m,snow_depth,wind_speed_10m,wind_speed_100m,pressure_msl,relative_humidity_2m,rainfall
0,2015-01-01,-4.404,2.446,-1.819000,-7.618870,0.0,57.0,-13.166501,0.00,16.010168,26.437830,1019.634949,42.515385,0.0
1,2015-01-02,-0.654,5.296,1.206417,-4.072746,0.0,1208.0,-7.004000,0.00,15.702933,28.147974,1020.354187,54.837727,0.0
2,2015-01-03,-3.254,3.846,-0.285250,-4.296842,10.2,2343.0,-3.541500,0.02,9.848926,19.898188,1032.354126,79.118011,10.2
3,2015-01-04,4.596,13.396,8.787666,6.437127,11.1,2400.0,8.200167,0.02,13.158006,26.931124,1014.125000,96.192757,11.1
4,2015-01-05,-1.454,12.146,3.512667,-1.893868,2.4,1258.0,-4.360250,0.00,20.636959,34.627483,1015.208313,60.083797,2.4


In [159]:
df_stations = pd.read_parquet('../data/02_intermediate/stations.parquet')
df_stations.head()

,id,name,latitude,longitude,tile,latitude_tile,longitude_tile,distance_to_tile_center
0,147,'Greenwich St Warren St',40.715422,-74.011220,1,40.711147,-74.010095,0.484747
1,152,'Warren St Church St',40.714740,-74.009106,1,40.711147,-74.010095,0.408147
2,173,'Broadway W 49 St',40.760683,-73.984527,3,40.754539,-73.986911,0.712126
3,224,'Spruce St Nassau St',40.711464,-74.005524,1,40.711147,-74.010095,0.386863
4,225,'W 14 St The High Line',40.741951,-74.008030,0,40.730239,-73.998749,1.519095


In [160]:
df_bike_turnover = pd.read_parquet('../data/02_intermediate/bike_turnover.parquet')
df_bike_turnover.head()

,station_id,date_ref,n_bikes_out,n_distinct_bikes_out,n_bikes_in,n_distinct_bikes_in,turnover,turnover_distinct
0,147,2016-03-30,0.0,0.0,2.0,1.0,1.0,1.0
1,152,2017-02-23,0.0,0.0,2.0,1.0,1.0,1.0
2,152,2017-02-24,0.0,0.0,2.0,1.0,1.0,1.0
3,152,2017-02-27,0.0,0.0,2.0,1.0,1.0,1.0
4,152,2017-02-28,0.0,0.0,2.0,1.0,1.0,1.0


In [161]:
df_stations = pd.read_parquet('../data/02_intermediate/stations.parquet')
df_stations.head()

,id,name,latitude,longitude,tile,latitude_tile,longitude_tile,distance_to_tile_center
0,147,'Greenwich St Warren St',40.715422,-74.011220,1,40.711147,-74.010095,0.484747
1,152,'Warren St Church St',40.714740,-74.009106,1,40.711147,-74.010095,0.408147
2,173,'Broadway W 49 St',40.760683,-73.984527,3,40.754539,-73.986911,0.712126
3,224,'Spruce St Nassau St',40.711464,-74.005524,1,40.711147,-74.010095,0.386863
4,225,'W 14 St The High Line',40.741951,-74.008030,0,40.730239,-73.998749,1.519095


In [162]:
df_bike = pd.read_parquet('../data/01_raw/bike.parquet')
df_bike.head()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender,trip_duration_in_min
0,376,2015-10-01 00:16:26,2015-10-01 00:22:42,3212,'Christ Hospital',40.734786,-74.050444,3207,'Oakland Ave',40.737604,-74.052478,24470,Subscriber,1960.0,1,6
1,739,2015-10-01 00:27:12,2015-10-01 00:39:32,3207,'Oakland Ave',40.737604,-74.052478,3212,'Christ Hospital',40.734786,-74.050444,24481,Subscriber,1960.0,1,12
2,2714,2015-10-01 00:32:46,2015-10-01 01:18:01,3193,'Lincoln Park',40.724605,-74.078406,3193,'Lincoln Park',40.724605,-74.078406,24628,Subscriber,1983.0,1,45
3,275,2015-10-01 00:34:31,2015-10-01 00:39:06,3199,'Newport Pkwy',40.728745,-74.032108,3187,'Warren St',40.721124,-74.038051,24613,Subscriber,1975.0,1,5
4,561,2015-10-01 00:40:12,2015-10-01 00:49:33,3183,'Exchange Place',40.716247,-74.033459,3192,'Liberty Light Rail',40.711242,-74.055701,24668,Customer,1984.0,0,9


In [ ]:
# spine is defined as 

def build_spine(df_bike):
    all_stations = list(set(
        df_bike['start_station_id'].unique().tolist() +
        df_bike['end_station_id'].unique().tolist()
    ))
    all_date_refs = pd.date_range(start=df_bike['start_time'].min(), end=df_bike['start_time'].max(), freq='D')

    df_spine = pd.DataFrame([
        [station, date_ref]
        for station in all_stations
        for date_ref in all_date_refs
    ], columns=['station_id','date_ref'])

    return df_spine

build_spine = build_spine(df_bike)

In [ ]:
def eda(df_bike):
    df_bike = (
         df_bike
        .assign(
            week_id = lambda x: x['start_time'].dt.to_period('W').dt.to_timestamp(how='start'),
            date_ref = lambda x: x['start_time'].dt.to_period('M').dt.to_timestamp(how='start'),
        )
        .eval('weekday = start_time.dt.dayofweek')
        .eval('age = date_ref.dt.year - birth_year')
        .assign(age=lambda x:pd.qcut(x['age'],10))
    )
    for var in ['gender','user_type','age','weekday']:
        df = (
            df_bike
            .pivot_table(index='date_ref', columns=var, values='bike_id', aggfunc='count')
        )
        df.div(df.sum(axis=1), axis=0).plot(kind='bar',stacked=True, figsize=(12,3), title='month x {}'.format(var))
eda(df_bike)

In [ ]:
class FPercentile:
    def __init__(self, percentile):
        self.percentile = percentile

    def __call__(self, s):
        return np.percentile(s, 99)

    def __str__(self):
        return f"p{self.percentile}"

fp99 = FPercentile(99)
fp05 = FPercentile(5)

In [ ]:
feature_engineering_dict = {
    'duration':['mean','min','sum'],
    'distance':['mean','median',fp05,fp99],
    'gender':['mean'],
    'isSubscriber':['mean'],
    'loopback':['mean'],
    'age_00_24':['mean'],
    'age_25_34':['mean'],
    'age_35_44':['mean'],
    'age_45_00':['mean'],
    'age':['mean','median',fp05,fp99],
}

################################
# This is the spine with features
################################

df_station_day_level = (
    df_bike
    .assign(
        week_id = lambda x: x['start_time'].dt.to_period('W').dt.to_timestamp(how='start'),
        date_ref = lambda x: x['start_time'].dt.to_period('D').dt.to_timestamp(how='start'),
    )
    # basic feature engineering at hide level
    .eval('distance = @haversine_distance(start_station_latitude, start_station_longitude, end_station_latitude, end_station_longitude)')
    .eval('gender = gender==1')
    .eval('loopback = start_station_id == end_station_id')
    .eval('isSubscriber = user_type=="Subscriber"')
    .eval('age = date_ref.dt.year - birth_year')
    .eval('age_00_24 = age <= 24')
    .eval('age_25_34 = age > 24 and age <= 34')
    .eval('age_35_44 = age > 34 and age <= 44')
    .eval('age_45_00 = age > 44')
    .rename(columns={'trip_duration_in_min':'duration'})

    # aggreating on day and start_station level
    .groupby(['start_station_id','date_ref'])
    .agg(
        **{'n_rents':('bike_id','size'),},
        **{
            f'{v}_{func}':(v,func)
                for v in feature_engineering_dict.keys()
                for func in feature_engineering_dict[v]
        }
    )
    .reset_index()
    .eval('weekday = date_ref.dt.weekday')
    .eval('month = date_ref.dt.month')
    .rename(columns={
        'isSubscriber_mean':'subscriber_rate',
        'gender_mean':'gender_rate',
        'start_station_id':'station_id',
    })
)

In [ ]:
df_station_day_level.head().T

In [ ]:
df_master_table = (
    df_station_day_level
    .merge(
        df_stations[['id','tile','distance_to_tile_center']]
        .rename(columns={'id':'station_id'})
        .astype({'tile':'str'}),
        on='station_id'
    )
    .merge(
        df_weather_by_day
        .rename(columns={'date':'date_ref'}),
        on='date_ref'
    )
)

In [ ]:
def window_feature(df, cols, metrics=['mean','sum'], add_trend=True, window=[1,2,3,7,15,30,60,90]):
    """
    Create a window feature for the given column in the dataframe.
    """

    for col in cols:
        for m in metrics:
            for w in window:
                df[f'{col}_{m}_w{w}'] = df[col].ffill().shift(w).agg(m)

        if 
    add_trend

    return df

In [ ]:
df_master_table.columns

In [ ]:
var_for_window =[
    'n_rents',
    'duration_mean',
    'duration_sum',
    'distance_mean',
    'distance_median',
    'distance_p5',
    'distance_p99',
    'gender_rate',
    'subscriber_rate',
    'loopback_mean',
    'age_00_24_mean',
    'age_25_34_mean',
    'age_35_44_mean',
    'age_45_00_mean',
    'age_mean',
    'age_median',
    'temperature_2m_min',
    'temperature_2m_max', 
    'temperature_2m',
    'apparent_temperature',
    'precipitation', 
    'cloud_cover', 
    'dew_point_2m',
    'snow_depth',
    'wind_speed_10m',
    'wind_speed_100m',
    'pressure_msl',
    'relative_humidity_2m',
    'rainfall',
    ]
df_master_table_with_window = (
    df_master_table
    .sort_values(['station_id','date_ref'])
    .groupby('station_id')
    .apply(
        window_feature,
        cols=var_for_window
    )
    .reset_index(drop=True)
)

In [ ]:
(
    df_master_table_with_window.isna()
    .groupby(df_master_table_with_window['date_ref'].dt.strftime('%Y%m'))
    .mean()
    .assign(max_missing=lambda x: x.max(axis=1))
    .query('max_missing > 0')
    .T
    .assign(max_missing=lambda x: x.max(axis=1))
    .query('max_missing > 0')
    .drop(columns=['max_missing'])
    .T
    # .sort_values('nmiss', ascending=False)
)